# A Full Training

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_dataset = load_dataset('glue', 'mrpc')
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation = True)

tokenized_dataset = raw_dataset.map(tokenize_function, batched = True)

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

### Prepare for training

- Remove non-required columns: sentence1, sentence2, idx
- Rename 'label' to 'labels'
- Set dataset format to PyTorch so it returns a PyTorch tensor instead of a list

In [4]:
tokenized_dataset = tokenized_dataset.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_dataset = tokenized_dataset.rename_column('label', 'labels')
tokenized_dataset.set_format('torch')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [5]:
from torch.utils.data import DataLoader

train_data = DataLoader(tokenized_dataset['train'], shuffle = True, batch_size = 8, collate_fn = data_collator)
test_dataset = DataLoader(tokenized_dataset['test'], batch_size = 8, collate_fn = data_collator)
eval_dataset = DataLoader(tokenized_dataset['validation'], batch_size = 8, collate_fn = data_collator)

In [6]:
# Checking dataset
for batch in train_data:
  print({k: v.shape for k,v in batch.items()})
  break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 53]), 'token_type_ids': torch.Size([8, 53]), 'attention_mask': torch.Size([8, 53])}


In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
output = model(**batch)
print('loss: ', output.loss,'\nShape:',output.logits.shape)

loss:  tensor(0.5676, grad_fn=<NllLossBackward0>) 
Shape: torch.Size([8, 2])


In [9]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr =5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs* len(train_data)
lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps =0,
    num_training_steps = num_training_steps
)

In [11]:
print(num_training_steps)

1377


### Training Loop

In [13]:
import torch
device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [14]:
print(device)

cuda


In [18]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [21]:
from tqdm.auto import tqdm
import evaluate

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
  model.train()
  for batch in train_data:
    batch ={k: v.to(device) for k,v in batch.items()}
    output = model(**batch)
    loss = output.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  metric = evaluate.load('glue', 'mrpc')
  model.eval()
  for batch in eval_dataset:
    batch = {k: v.to(device) for k,v in batch.items()}
    with torch.no_grad():
      output = model(**batch)
    logits = output.logits
    prediction = torch.argmax(logits, dim =-1)
    metric.add_batch(predictions = prediction, references = batch['labels'])
  eval_score = metric.compute()
  print(f'validation at epoch {epoch}: {eval_score}')

  0%|          | 0/1377 [00:00<?, ?it/s]

validation at epoch 0: {'accuracy': 0.8627450980392157, 'f1': 0.903448275862069}
validation at epoch 1: {'accuracy': 0.8627450980392157, 'f1': 0.903448275862069}
validation at epoch 2: {'accuracy': 0.8627450980392157, 'f1': 0.903448275862069}
